# Vehicle Cut-In Detection

This notebook is made by Ved Verma.

In this notebook I shall explain the code and how to use the code for your own purposes

## Approach

Essentially what we do is initialize a window between two lines. The distance between the two lines will be known (In our case it is fixed to 10m).

What we do is check if any vehicle is whithin this window. If yes, then we use the speed using the data provided from the OBD (On-Board Diagnostic) to calculate a TTC (Time To Collision).

If the TTC falls under 0.7s, we initiate a warning.

## Importing Necessary Packages

This code section imports all the required packages for this project.
Make sure you have installed all the packages.

In [11]:
import cv2
import pandas as pd
from ultralytics import YOLO
from ultralytics.solutions import speed_estimation
from tracker import Tracker

Chooses the YOLO model and uses the pre-trained weights.

In [12]:
model = YOLO('yolov8s.pt')

This function tracks the mouse movement and prints the (x, y) coordinates when the mouse cursor is moved within the OpenCV window.

In [13]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        colorsBGR = [x, y]
        print(colorsBGR)

In [14]:
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

This code segment initializes a video capture object using OpenCV which allows us to read frames from the video file.

In [15]:
cap = cv2.VideoCapture('0000000_1.mp4')  #Change to the path of the input video file

This code opens the `classes.txt` which contains all the classes of objects that can be detected using the YOLOv8 model.

In [16]:
my_file = open("classes.txt", "r")
data = my_file.read()
class_list = data.split("\n")

This segment initializes all the constants.

In [17]:
fps = cap.get(cv2.CAP_PROP_FPS) #Gets the FrameRate of the video
count = 0 
tracker = Tracker() # Gets the tracker function to allow us to track the vehicles
cy1 = 354 # Defines the y-coordinates of the line where we detect a cut in and collision
cy2 = 412
offset = 6

This segment initializes an video file `output.mp4` which is the output of the project.

In [18]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (1020, 500))

This function calculates the time to collision over the 10m window defined by the lines. If the time to collision is less than 0.7s we trigger a warning.

In [19]:
def check_warning(frame, cx, cy, vehicle_id, x, y):
    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
    distance = 10
    speed = 15
    if (distance / speed) < 0.7:
        cv2.putText(frame, f'WARNING: COLLISION DETECTED', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

This code processes video frames, detects vehicles, tracks them, and displays warnings based on their position.

In [20]:
while True:
    # Read the next frame
    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if no more frames

    # Process every third frame
    if count % 3 != 0:
        continue

    # Resize frame to 1020x500
    frame = cv2.resize(frame, (1020, 500))

    # Perform object detection (results from model)
    results = model.predict(frame)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")

    # Extract bounding boxes for vehicles
    vehicle_list = []
    for index, row in px.iterrows():
        x1, y1, x2, y2, _, d = map(int, row)
        c = class_list[d]
        if c in ['car', 'truck', 'bus', 'motorcycle']:
            vehicle_list.append([x1, y1, x2, y2])

    # Update tracker with bounding box list
    bbox_id = tracker.update(vehicle_list)

    # Draw lines for regions of interest
    cv2.line(frame, (274, cy1), (814, cy1), (255, 255, 255), 1)
    cv2.line(frame, (177, cy2), (927, cy2), (255, 255, 255), 1)

    # Process each tracked vehicle
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = (x3 + x4) // 2
        cy = (y3 + y4) // 2

        # Draw rectangle around vehicle
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 1)

        # Display "Vehicle Detected" label
        cv2.putText(frame, f'Vehicle Detected', (x4, y4),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 255), 2)

        # Check if vehicle is within vertical range
        if cy2 < (cy + offset) and cy2 > (cy - offset):
            check_warning(frame, cx, cy, id, x4, y4)

    # Write processed frame to output video
    out.write(frame)

    # Display frame
    cv2.imshow("RGB", frame)

    # Exit loop if 'Esc' key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Cleanup: Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


0: 320x640 4 persons, 2 cars, 3 motorcycles, 1 truck, 254.3ms
Speed: 2.0ms preprocess, 254.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 car, 2 motorcycles, 249.3ms
Speed: 3.0ms preprocess, 249.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 persons, 3 cars, 5 motorcycles, 2 trucks, 253.3ms
Speed: 2.0ms preprocess, 253.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 2 cars, 2 motorcycles, 1 truck, 255.3ms
Speed: 3.0ms preprocess, 255.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 car, 3 motorcycles, 1 truck, 275.3ms
Speed: 2.0ms preprocess, 275.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 1 car, 3 motorcycles, 1 truck, 259.8ms
Speed: 2.0ms preprocess, 259.8ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 2 cars, 1 motorcycle, 

## END